In [1]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers,datasets

## 0.前言

Keras库分为前段和后段：
* 后端调用现有的深度学习框架实现底层运算，如Theano、CNTK、TensorFlow
* 前端指前端接口，是Keras抽象过的一组统一接口函数。

在TensorFlow 2版本中，Keras已被实现在tf.keras子模块中，并且只以TensorFlow为后端做运算。

Keras提供了一系列高层的神经网络相关类和函数，如经典数据集加载函数、网络层类、模型容器、损失函数类、优化器类、经典模型类等。

## 1.管理数据集
对于经典数据集，通过一行代码即可下载、管理、加载数据集，这些数据集包括Boston房价预测数据集、CIFAR图片数据集、MNIST/FashionMNIST手写数字图片数据集、IMDB文本数据集等。

关于数据集的加载、管理的实例可见[a05.07-经典数据集加载.ipynb](https://nbviewer.jupyter.org/github/zfhxi/Learn_tensorflow/blob/master/ch05-TensorFlow%E8%BF%9B%E9%98%B6/a05.07-%E7%BB%8F%E5%85%B8%E6%95%B0%E6%8D%AE%E9%9B%86%E5%8A%A0%E8%BD%BD.ipynb)
<br>总结步骤为如下（以mnist数据集为例）：

In [2]:
'''
1.数据加载
'''
# 加载MNIST数据集
(x,y),(x_test,y_test)=datasets.mnist.load_data()
print(f'x: {x.shape}\ny: {y.shape}\nx_test: {x_test.shape}\ny_test: {y_test.shape}')

# 转换成Dataset对象
train_db = tf.data.Dataset.from_tensor_slices((x, y)) # 构建 Dataset 对象
print(train_db)

'''
2.随机打散
'''
train_db=train_db.shuffle(buffer_size=10000) # 随机打散样本，不会打乱样本与标签映射关系
print(train_db)

'''
3.批训练
'''
train_db=train_db.batch(128) # 设置批训练，batch size为128
print(train_db)

'''
4.预处理
'''
def preprocess(x,y):
    # 调用此函数会自动传入x，y
    # 标准化到0~1
    x=tf.cast(x,dtype=tf.float32)/255.
    x=tf.reshape(x,[-1,28*28]) # 打平
    y=tf.cast(y,dtype=tf.int32) # 转换成整型张量
    y=tf.one_hot(y,depth=10) # 进行one-hot编码
    return x,y
train_db=train_db.map(preprocess)
print(train_db)

'''
5.循环训练
'''
epochs=1
def do(x,y):
    pass

for step,(x,y) in enumerate(train_db):
    do(x,y)
# or
for x,y in train_db:
    do(x,y)
# generally
for epoch in range(epochs):
    for step,(x,y) in enumerate(train_db):
        do(x,y)


x: (60000, 28, 28)
y: (60000,)
x_test: (10000, 28, 28)
y_test: (10000,)
<TensorSliceDataset shapes: ((28, 28), ()), types: (tf.uint8, tf.uint8)>
<ShuffleDataset shapes: ((28, 28), ()), types: (tf.uint8, tf.uint8)>
<BatchDataset shapes: ((None, 28, 28), (None,)), types: (tf.uint8, tf.uint8)>
<MapDataset shapes: ((None, 784), (None, 10)), types: (tf.float32, tf.float32)>


回顾之前加载、管理数据集的操作：

**a03.01**

In [3]:
(x,y),(x_val,y_val)=datasets.mnist.load_data()

# 转换为tensor格式并分批
x=tf.convert_to_tensor(x,dtype=tf.float32)/255.
y=tf.convert_to_tensor(y,dtype=tf.int32)
y=tf.one_hot(y,depth=10)
print(x.shape)
print(y.shape)
train_dataset=tf.data.Dataset.from_tensor_slices((x,y))
train_dataset=train_dataset.batch(200)

(60000, 28, 28)
(60000, 10)


**a04.10**

In [4]:
def load_data():
    # 加载MNIST
    (x,y),(x_val,y_val)=datasets.mnist.load_data()
    # 转换为浮点张量，并缩放到（0～1）
    x=tf.convert_to_tensor(x,dtype=tf.float32)/255.
    # 转换为整型张量
    y=tf.convert_to_tensor(y,dtype=tf.int32)
    # one-hot编码
    y=tf.one_hot(y,depth=10)
    # 改变视图
    x=tf.reshape(x,[-1,28*28])

    # 构建数据集对象
    train_dataset=tf.data.Dataset.from_tensor_slices((x,y))
    #批量训练
    train_dataset=train_dataset.batch(200)
    return train_dataset

**a05.08**

In [5]:
def preprocess(x,y):
    # 调用此函数会自动传入x，y
    # 标准化到0~1
    x=tf.cast(x,dtype=tf.float32)/255.
    x=tf.reshape(x,[-1,28*28]) # 打平
    y=tf.cast(y,dtype=tf.int32) # 转换成整型张量
    y=tf.one_hot(y,depth=10) # 进行one-hot编码
    return x,y

def load_data():
    # 加载MNIST
    (x,y),(x_val,y_val)=datasets.mnist.load_data()
    batchsz=512
    # 构建数据集对象
    train_dataset=tf.data.Dataset.from_tensor_slices((x,y))
    train_dataset=train_dataset.shuffle(1000)
    #批量训练
    train_dataset=train_dataset.batch(batchsz)
    train_dataset=train_dataset.map(preprocess)
    train_dataset=train_dataset.repeat(20)

    # 加载验证/测试集
    val_dataset=tf.data.Dataset.from_tensor_slices((x_val,y_val))
    val_dataset=val_dataset.shuffle(1000).batch(batchsz).map(preprocess)
    return train_dataset,val_dataset

**a06.08**
```python
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import os
# dataset_path=keras.utils.get_file('auto-mpg.data','file://'+os.path.abspath('./data/auto-mpg.data'))
# 下载汽车效能数据集
dataset_path=keras.utils.get_file('auto-mpg.data','http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data')
print(dataset_path)
# 利用pandas读取数据集
column_names=['MPG','Cylinders','Displacement','Horsepower','Weight','Acceleration','Model Year','Origin']
raw_dataset=pd.read_csv(dataset_path,names=column_names,na_values='?',comment='\t',sep=" ",skipinitialspace=True)
dataset=raw_dataset.copy()
# 查看部分数据
print(dataset.head())

# 删除空字段数据项（含有空字段的行）
print(dataset.isna().sum()) # 统计空白行
dataset=dataset.dropna() # 删除空白数据项
print(dataset.isna().sum()) # 再次统计空白行

# 先弹出（删除并返回）origin这一列
origin=dataset.pop('Origin')
# 根据origin列来写入新的3个列
dataset['USA']=(origin==1)*1.0
dataset['Europe']=(origin==2)*1.0
dataset['Japan']=(origin==3)*1.0
print(dataset.tail()) # 查看表格后几项

# 按照8:2比例划分训练/测试集
train_dataset=dataset.sample(frac=.8,random_state=0)
test_dataset=dataset.drop(train_dataset.index)

# 将MPG字段作为标签，因此从train_dataset/test_dataset中移出
train_labels=train_dataset.pop('MPG')
test_labels=test_dataset.pop('MPG')

# 统计训练集的各个字段的均值和标准差，并完成数据的标准化
# 查看训练集的输入X的统计数据
train_stats=train_dataset.describe()
# print(train_stats)
import sys
try:
    train_stats.pop('MPG')
    # print(train_stats)
except Exception:
    pass
# 标准化数据
def norm(x,stats):
    '''
    标准化，减去每个字段的均值，并除以标准差
    :param x:
    :return:
    '''
    return (x-stats.loc['mean'])/stats.loc['std']
normed_train_data=norm(train_dataset,train_stats) # 标准化训练集
normed_test_data=norm(test_dataset,train_stats) # 标准化测试集
print(normed_train_data.shape)
print(normed_test_data.shape)
print(train_labels.shape)
print(test_labels.shape)

# 利用切分的训练集数据构建数据集对象：
train_db=tf.data.Dataset.from_tensor_slices(
    (normed_train_data.values,train_labels.values)) # 构建Dataset对象
train_db=train_db.shuffle(100).batch(32) # 随机打散，批量化
```

## 2.常见网络层

`tf.nn`模块中实现了常见网络层的接口函数。如`tf.nn.softmax`<br>
`tf.keras.layers`提供了大量常见网络层的类，如全连接层、激活函数层、池化层、卷积层、循环神经网络层等

更多见<https://tensorflow.google.cn/api_docs/python/tf/keras/layers/>

例如Softmax层的创建：

In [6]:
%reset -f
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
x=tf.constant([2.,1.,.1])
layer=layers.Softmax(axis=-1) # 创建Softmax层
out=layer(x) # 调用Softmax的__call__进行前向计算
print(out)

out2=tf.nn.softmax(x)
print(out2)

tf.Tensor([0.6590012 0.242433  0.0985659], shape=(3,), dtype=float32)
tf.Tensor([0.6590012 0.242433  0.0985659], shape=(3,), dtype=float32)


**Input**

用于实例化一个Keras张量，充当类似一个占位符，可以用来限定每个输入样本的特征形状
```python
# tf.keras.Input( or
tf.keras.layers.Input(
    shape=None,
    batch_size=None,
    name=None,
    dtype=None,
    sparse=False,
    tensor=None,
    **kwargs
)
```

In [7]:
X=tf.keras.layers.Input(shape=(32,)) # 限定每个样本特征维度长度为32
y=tf.square(X)
print(X)
print(y)

Tensor("input_1:0", shape=(None, 32), dtype=float32)
Tensor("Square:0", shape=(None, 32), dtype=float32)


**Dense**

常用的全连接层
```python
tf.keras.layers.Dense(
    units, activation=None, use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros', kernel_regularizer=None,
    bias_regularizer=None, activity_regularizer=None, kernel_constraint=None,
    bias_constraint=None, **kwargs
)
```

In [8]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(16,))) # 限定每个样本的特征长度为16
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(32))
print(model.output_shape)
X=tf.random.normal([4,16])
print(X.shape)
print(model(X).shape)

(None, 32)
(4, 16)
(4, 32)


**Conv1D**

一维卷积，常用作时间卷积
```python
tf.keras.layers.Conv1D(
    filters, kernel_size, strides=1, padding='valid',
    data_format='channels_last', dilation_rate=1, groups=1,
    activation=None, use_bias=True, kernel_initializer='glorot_uniform',
    bias_initializer='zeros', kernel_regularizer=None,
    bias_regularizer=None, activity_regularizer=None, kernel_constraint=None,
    bias_constraint=None, **kwargs
)
```

In [9]:
input_shape = (4, 10, 128) # 4个样本，10个时间刻度，128维度的向量
x = tf.random.normal(input_shape)
y = tf.keras.layers.Conv1D(
    32, 3, activation='relu',input_shape=input_shape[1:])(x)
print(y.shape)

(4, 8, 32)


**Conv2D**

2D卷积，常用作图像上的空域卷积
```python
tf.keras.layers.Conv2D(
    filters, kernel_size, strides=(1, 1), padding='valid',
    data_format=None, dilation_rate=(1, 1), groups=1, activation=None,
    use_bias=True, kernel_initializer='glorot_uniform',
    bias_initializer='zeros', kernel_regularizer=None,
    bias_regularizer=None, activity_regularizer=None, kernel_constraint=None,
    bias_constraint=None, **kwargs
)
```

In [10]:
input_shape = (4, 28, 28, 3) # 4张28x28的RGB图片
x = tf.random.normal(input_shape)
y = tf.keras.layers.Conv2D(
    2, 3, activation='relu', input_shape=input_shape[1:])(x)
print(y.shape)

(4, 26, 26, 2)


**ReLU**

```python
tf.keras.layers.ReLU(
    max_value=None, negative_slope=0, threshold=0, **kwargs
)
```
默认计算规则是`max(x,0)`，否则：

* `f(x) = max_value if x >= max_value`
* `f(x) = x if threshold <= x < max_value`
* `f(x) = negative_slope * (x - threshold) otherwise`
```

In [11]:
layer = tf.keras.layers.ReLU()
output = layer([-3.0, -1.0, 0.0, 2.0])
print(list(output.numpy()))

layer = tf.keras.layers.ReLU(max_value=1.0)
output = layer([-3.0, -1.0, 0.0, 2.0])
print(list(output.numpy()))

layer = tf.keras.layers.ReLU(threshold=1.5)
output = layer([-3.0, -1.0, 1.0, 2.0])
print(list(output.numpy()))

layer = tf.keras.layers.ReLU(threshold=0.5,max_value=1.5)
output = layer([-3.0, -1.0, 1.0, 2.0])
print(list(output.numpy()))

layer = tf.keras.layers.ReLU(negative_slope=2.0)
output = layer([-3.0, -1.0, 0.0, 2.0])
print(list(output.numpy()))

[0.0, 0.0, 0.0, 2.0]
[0.0, 0.0, 0.0, 1.0]
[-0.0, -0.0, 0.0, 2.0]
[0.0, 0.0, 1.0, 1.5]
[-6.0, -2.0, 0.0, 2.0]


## 3.网络容器
对于常见的网络，需要手动调用每一层的类实例完成前向传播运算，当网络层数变得较深时，这一部分代码显得非常臃肿。可以通过 Keras 提供的网络容器 Sequential 将多个网络层封装成一个大网络模型，只需要调用网络模型的实例一次即可完成数据从第一层到最末层的顺序传播运算。

如：

In [12]:
# 方式1
network=tf.keras.Sequential([ # 封装为一个网络
    layers.Input(shape=(4,)),
    layers.Dense(3,activation=None),
    layers.ReLU(),
    layers.Dense(2,activation=None),
    layers.ReLU(),
])
print(network.output_shape)
network.summary()
try:
    x=tf.random.normal([8,3])
    print(network(x).shape)
except Exception as e:
    print(e)
    x=tf.random.normal([8,4])
    print(network(x).shape)

# 方式2
network1=tf.keras.Sequential([]) # 创建空的网络容器
layers_num=2
for _ in range(layers_num):
    network1.add(layers.Dense(3))
    network1.add(layers.ReLU())
network1.build(input_shape=(None,5)) # 指定网络权重维度
network1.summary()
x=tf.random.normal([8,5])
print(network1(x).shape)

# 打印待优化参数
for p in network1.trainable_variables:
    print(f'name: {p.name}\tshape: {p.shape}')


(None, 2)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 3)                 15        
_________________________________________________________________
re_lu_5 (ReLU)               (None, 3)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 8         
_________________________________________________________________
re_lu_6 (ReLU)               (None, 2)                 0         
Total params: 23
Trainable params: 23
Non-trainable params: 0
_________________________________________________________________
Matrix size-incompatible: In[0]: [8,3], In[1]: [4,3] [Op:MatMul]
(8, 2)
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)     

In [ ]:
import os
pid=os.getpid()
!kill -9 $pid

